## Dependencies

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [14]:
# Now, load the downloaded data into pandas
df = pd.read_csv("Alaska.csv")

# Let us inspect the columns
print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'Alaska.csv'

In [6]:
# Then, we are going to filter the data
# Filter by region
df_filt = df[df["o1region"] == 1]

pca_cols = [
    "zmin_m", "zmax_m", "zmed_m", "zmean_m",
    "slope_deg", "aspect_deg", "lmax_m"
]

df_filt = df.dropna(subset=pca_cols)

# Combine Filters
df_filt = (
    df[df["o1region"] == 1]
    .dropna(subset=pca_cols)
)


NameError: name 'df' is not defined

In [7]:
X = df_filt[pca_cols]

print(X.dtypes)


zmin_m        float64
zmax_m        float64
zmed_m        float64
zmean_m       float64
slope_deg     float64
aspect_deg    float64
lmax_m          int64
dtype: object


In [ ]:
# First, we extract Glacier data and drop rows with NaN values
glacier = Alaska[slope_deg].dropna()

# Let's center the data by subtracting the mean
# Note that we do not *normalize* the data
glacier_centered = glacier - glacier.mean()